# *Predicting Length of Stay at a Hospital*

Task is to predict Length of Stay (LOS) Category from the below data. Increased importance to accurately patients with higher LOS.

In [116]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical 


train_data=pd.read_csv('/kaggle/input/av-healthcare-analytics-ii/healthcare/train_data.csv')
test_data=pd.read_csv('/kaggle/input/av-healthcare-analytics-ii/healthcare/test_data.csv')

In [117]:
print(train_data.shape)
print(train_data.head())
print(test_data.shape)

# **Data Exploration**

In [118]:
#  view the dataset's categories at-a-glance
train_data.info()

In [119]:
test_data.info()

In [120]:
# visualize Length of Stay intervals only
plt.figure(figsize = (15,10))
sns.countplot(y = 'Stay', data = train_data, palette = 'Set3')
plt.xlabel('Admissions', size = 25)
plt.ylabel('Length of Stay (in days)', size = 25)
plt.title('Number of Admissions per Length of Stay(n = {})'.format(len(train_data)))
plt.show()

In [121]:
# visualize distribution of LOS by age cohort
los_indices = train_data.Stay.value_counts().index[:11]
age_indices = train_data.Age.value_counts().index[:10]
cross_data = train_data[train_data.Stay.isin(los_indices)& (train_data.Age.isin(age_indices))]
plt.figure()
cross_table = pd.crosstab(columns= cross_data.Stay, index = cross_data.Age) 
cross_table.plot.bar(figsize = (10,10), title="Stay Lengths per Age cohort")
plt.xlabel('Age cohorts', size = 20)
plt.ylabel('Admissions', size = 20)
plt.show()

In [122]:
# visualize admissions by departments
plt.figure(figsize=(15, 10))
sns.countplot(x = train_data.Department)
plt.title("Admission Counts for Departments in the Dataset")
plt.ylabel('Admission counts', size = 20)
plt.xlabel('Departments', size = 20)

In [123]:
# visualize admissions by illness severity
f, (vis1, vis2) = plt.subplots(1,2, figsize = (14,6))

# +++++++++
sns.countplot(x= train_data["Severity of Illness"], ax = vis1, palette = 'Set1')
vis1.set_title("""Distribution of Severity of Illness in training dataset
(n = {})""".format(len(train_data)))
vis1.set_ylabel("Total Admissions")

# ++++++++++
illness = train_data.groupby(["Severity of Illness", "Stay"]).size()
illness = (illness/illness.groupby(level=1).sum()).reset_index()
# ++++++++++++++++++

sns.lineplot(x = 'Stay', y = 0, hue = "Severity of Illness", data = illness, ax = vis2, palette = 'Set2')
vis2.set_title("""Proportion of Length of Stay to Severity of Illness
(n = {})""".format(len(train_data)))
vis2.set_ylabel("Percent of Admissions")

plt.tight_layout()
plt.show()

In [124]:
# correlation matrix
corr_matrix = train_data.corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr_matrix,vmax=.4, square=True,annot=True)

In [125]:
print(train_data.dtypes.value_counts())

In [126]:
print(train_data.describe(include='int').T)

In [127]:
print(train_data.describe(include='float').T)

In [128]:
print(train_data.describe(include='object').T)

# **Data Cleaning**

In [129]:
# remove null values
train_data = train_data.dropna()
test_data = test_data.dropna()
# case ids will never help with predicting,so this category is removed;
train_data = train_data.drop(['case_id'], axis = 1)
test_data = test_data.drop(['case_id'], axis = 1)

In [130]:
# folds in repeated patient ids instances  into N number of visits 
train_data['Visits'] = train_data.groupby(['patientid'])['patientid'].transform('count')
train_data = train_data.drop(['patientid'], axis=1) 

test_data['Visits'] = test_data.groupby(['patientid'])['patientid'].transform('count')
test_data = test_data.drop(['patientid'], axis=1) 

In [131]:
# normalize some of the data in columns with high amount of unique values
# academic honesty: inspired by example by JAKE TURICCHI on Kaggle --> see code section for this kaggle dataset
threshold = 0.05
hospital_codes = train_data['Hospital_code'].value_counts(normalize = True)
hospital_codes_below_threshold = hospital_codes.where(hospital_codes < threshold).dropna().index.values
train_data['Hospital_code']= np.where(train_data['Hospital_code'].isin(hospital_codes_below_threshold), '0',  train_data['Hospital_code'])

city_code_patients = train_data['City_Code_Patient'].value_counts(normalize = True)
patient_codes_below_threshold = city_code_patients.where(city_code_patients < threshold).dropna().index.values
train_data['City_Code_Patient']= np.where(train_data['City_Code_Patient'].isin(patient_codes_below_threshold), '0',  train_data['City_Code_Patient'])

# **Encoding**

In [132]:
# prepare columns to ease the encoding of categorical values
target=['Hospital_code', 'City_Code_Hospital',
        'City_Code_Patient', 'Severity of Illness',
        'Hospital_type_code', 'Hospital_region_code', 
        'Department', 'Type of Admission', 'Stay',
        'Ward_Facility_Code',  'Ward_Type', 'Age'] 
train_data[target] = train_data[target].astype('category')

# test data does not have Stay column
target_two=['Hospital_code', 'City_Code_Hospital',
        'City_Code_Patient', 'Severity of Illness',
        'Hospital_type_code', 'Hospital_region_code', 
        'Department', 'Type of Admission',
        'Ward_Facility_Code',  'Ward_Type', 'Age'] 

test_data[target_two] = test_data[target_two].astype('category')

In [133]:
# integer encoding
lbl_encoder = LabelEncoder()

train_data['Hospital_code_lbl']=lbl_encoder.fit_transform(train_data['Hospital_code'])
train_data['City_Code_Hospital_lbl']=lbl_encoder.fit_transform(train_data['City_Code_Hospital'])
train_data['City_Code_Patient_lbl']=lbl_encoder.fit_transform(train_data['City_Code_Patient'])
train_data['Illness_lbl']=lbl_encoder.fit_transform(train_data['Severity of Illness'])
train_data['Hospital_type_code_lbl']=lbl_encoder.fit_transform(train_data['Hospital_type_code'])
train_data['Hospital_region_code_lbl']=lbl_encoder.fit_transform(train_data['Hospital_region_code'])
train_data['Department_lbl']=lbl_encoder.fit_transform(train_data['Department'])
train_data['Admission_type_lbl']=lbl_encoder.fit_transform(train_data['Type of Admission'])
train_data['Ward_facility_lbl']=lbl_encoder.fit_transform(train_data['Ward_Facility_Code'])
train_data['Ward_type_lbl']=lbl_encoder.fit_transform(train_data['Ward_Type'])
#train_data['Age_lbl']=lbl_encoder.fit_transform(train_data['Age'])
#train_data['Stay_lbl']=lbl_encoder.fit_transform(train_data['Stay'])

# excluded Stay and Age because they are to be onehotencoded
train_data=train_data.drop(['Hospital_type_code','City_Code_Hospital', 
                            'Hospital_region_code','Ward_Type','Ward_Facility_Code', 
                            'City_Code_Patient', 'City_Code_Hospital' 
                            ], axis = 1) 

train_data = train_data.reset_index(drop = True)

test_labels =['Hospital_code', 'City_Code_Hospital',
        'City_Code_Patient', 'Severity of Illness',
        'Hospital_type_code', 'Hospital_region_code', 
        'Department', 'Type of Admission',
        'Ward_Facility_Code',  'Ward_Type',] 

# excluded Stay and Age because they are to be onehotencoded
for label in test_labels:
    test_data[label] = lbl_encoder.fit_transform(test_data[label])
    
    
test_data = test_data.reset_index(drop = True)

In [134]:
print(train_data.head())

In [135]:
def onehotencode(dataframe, column):
    df = dataframe.copy()
    dummies = pd.get_dummies(df[column])
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(column, axis = 1)
    return df

In [136]:
#indepedent_variable = train_data['Stay'].copy()
#train_data = onehotencode(train_data, 'Stay')
#train_data = onehotencode(train_data, 'Age')
#dependent_variables = train_data

#test_data = onehotencode(test_data, 'Age')

In [137]:
#scaler = StandardScaler()
#alread_numeric_columns=['Available Extra Rooms in Hospital', 'Visitors with Patient',
#              'Admission_Deposit', 'Visits']

#for column in train_data[alread_numeric_columns].columns:
#   train_data[column] = scaler.fit_transform(train_data[[column]])

In [138]:
train_data.head()

In [139]:
numpy_train_data = train_data.to_numpy()
numpy_train_labels =  train_data.values
print(type(train_data))
print(type(numpy_train_data))
print(type(numpy_train_labels))
print(numpy_train_data.shape)
print(numpy_train_labels.shape)

In [140]:
# use pd.set_option('max_columns', None) in console to allow for viewing of all columns
train_data

In [141]:
#train_data.loc[:,Stay]

In [142]:
train_labels =  train_data.values
train_labels

In [143]:
input_features = ["Severity of Illness", "Age", "Type of Admission",
                  "Hospital_code", "Department", "Visitors with Patient"]

y_feature = "Stay"
X_train = train_data.loc[:, input_features + [y_features]]

ordered_cats = input_features[0:2]
cats = input_features[2:-1]
num = [input_features[-1]]




In [144]:
def prepare_dataset():

    # Convert categorical variables to indices
    X_trn = None
    m = MinMaxScaler()

    for c in X_train.columns:
        #print(c)

        if c in cats or c in ordered_cats or c == y_feature:
            _ = pd.factorize(X_train[c], sort=True)[0]
            if c in cats:
                _ = to_categorical(_, num_classes=X_train[c].unique().size)
            elif c in ordered_cats:
                _ = _/np.max(_)
        else:
            #print(c)
            _ = m.fit_transform(X_train[c].values.reshape(-1,1))[:, 0]

        try:
            print("Xtrn: ", X_trn.shape) 
        except:
            pass
        print("_: ",_.shape)

        try:
            if len(_.shape) == 1:
                    print("1D")
                    X_trn = np.hstack((X_trn, _.reshape(-1,1)))
            else:
                X_trn = np.hstack((X_trn, _))  
        except Exception as e:
            print("Error")
            if len(_.shape) == 1:
                    print("1D")
                    X_trn = _.reshape(-1,1)
            else:
                X_trn = _

    X_trn = X_trn


    y_trn = X_trn[:, -1]
   
    print(y_trn)
    X_trn = X_trn[:, :-1]
   
    print(X_trn)

    # adding Visitors Number ^ 2 to add another feature
    #X_trn = np.vstack((X_trn.T, m.fit_transform(((X_train["Visitors with Patient"]**2).values).reshape(-1,1))[:, 0])).T

    #print(X_trn.shape)
    
    y_trn = to_categorical(y_trn, num_classes=train_data.Stay.unique().size)
    print(y_trn)
    X_trn, X_val, y_trn, y_val = train_test_split(X_trn, y_trn, test_size=0.3)
    
    return X_trn, X_val, y_trn, y_val

In [145]:
# random_state is set to my favorite seed number
X_train, X_test, y_train,y_test  = prepare_dataset()


In [146]:
print(X_train)
print(y_train)

In [147]:
# code from Chollet book
# def vectorize_sequences(sequences, dimension = 10000):
#     results = np.zeros((len(sequences), dimension))
#     for i, sequence in enumerate(sequences):
#         results[i, sequence] = 1
        
#     return results


# x_train = vectorize_sequences(train_data)
# x_test = vectorize_sequences(test_data)

In [148]:
train_data.Stay.unique().size


# **Training the Neural Network**

In [149]:
# define model to tackle this single-label, multiclass classification problem
model = Sequential()
model.add(Dense(128, input_dim = X_train.shape[1] , activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
# model.add(Dense(32, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(11, activation='softmax'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])



In [150]:

X_train=np.asarray(X_train).astype(np.int)

y_train=np.asarray(y_train).astype(np.int)

model.fit(X_train,y_train,validation_split = 0.2, batch_size = 225, epochs = 5, shuffle = True, verbose = 2)

# _, accuracy = model_base.evaluate(x_train, y_test, verbose=0)
# print('Accuracy: %.2f' % (accuracy*100))
# monitor = EarlyStopping(monitor = '??', min_delta = ?)
# checkpointer = ModelCheckPoint(filepath = 'DIR', verbose = 0, save_best_only= True)

In [ ]:
# create confusion matrix
# academic honesty: code taken from scikit learn's website